In [0]:
select * from `pricing-analytics`.bronze.daily_pricing limit 5;

In [0]:
desc `pricing-analytics`.bronze.daily_pricing

In [0]:
select 

DATE_OF_PRICING,
ROW_ID,
STATE_NAME,
MARKET_NAME,
PRODUCTGROUP_NAME,
PRODUCT_NAME,
VARIETY,
ORIGIN,
ARRIVAL_IN_TONNES,
MINIMUM_PRICE,
MAXIMUM_PRICE,
MODAL_PRICE,
source_file_load_date 

from 
`pricing-analytics`.bronze.daily_pricing limit 10

In [0]:
use catalog `pricing-analytics`;
create schema if not exists silver;
drop table if exists `pricing-analytics`.`silver`.`daily_pricing_silver`;
create table `pricing-analytics`.`silver`.`daily_pricing_silver`
(
DATE_OF_PRICING date,
ROW_ID bigint,
STATE_NAME string,
MARKET_NAME string,
PRODUCTGROUP_NAME string,
PRODUCT_NAME string,
VARIETY string,
ORIGIN string,
ARRIVAL_IN_TONNES decimal(18,2),
MINIMUM_PRICE decimal(36,2),
MAXIMUM_PRICE decimal(36,2),
MODAL_PRICE decimal(36,2),
SOURCE_FILE_LOAD_DATE timestamp,
LAKEHOUSE_INSERTED_DATE timestamp,
LAKEHOUSE_UPDATED_DATE timestamp
);

In [0]:

insert into `pricing-analytics`.`silver`.`daily_pricing_silver`
select 

to_date(DATE_OF_PRICING,'dd/mm/yyyy'),
cast(ROW_ID as bigint),
STATE_NAME,
MARKET_NAME,
PRODUCTGROUP_NAME,
PRODUCT_NAME,
VARIETY,
ORIGIN,
cast(ARRIVAL_IN_TONNES as decimal(18,2)),
cast(MINIMUM_PRICE as decimal(36,2)),
cast(MAXIMUM_PRICE as decimal(36,2)),
cast(MODAL_PRICE as decimal(36,2)),
source_file_load_date,
current_timestamp(),
current_timestamp()

from 
`pricing-analytics`.`bronze`.`daily_pricing`
where source_file_load_date > (select nvl(max(processed_table_datetime),'1900-01-01') from `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs` where process_name='daily_pricing_silver' and process_status='completed');


In [0]:
select * from `pricing-analytics`.`silver`.`daily_pricing_silver`;

In [0]:
desc `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs`;

In [0]:
select * from `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs`;

In [0]:
alter table `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs`
add column processed_table_datetime timestamp;

In [0]:
desc `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs`;

In [0]:
select 'daily_pricing_silver', max(source_file_load_date), 'completed' from `pricing-analytics`.`silver`.`daily_pricing_silver`;

In [0]:
insert into `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs`(process_name,processed_table_datetime,process_status)
select 'daily_pricing_silver', max(source_file_load_date), 'completed' from `pricing-analytics`.`silver`.`daily_pricing_silver`;

In [0]:
delete from `pricing-analytics`.`silver`.`daily_pricing_silver`;
delete from `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs` where process_name='daily_pricing_silver';
